In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

#database imports
import psycopg2
from datetime import datetime
import time

#for accessing env
import os 
from dotenv import load_dotenv

In [2]:
# loading trained model
model = tf.keras.models.load_model('model/EmotionEcho_v5.keras', compile=False)


In [3]:
#database script

# Database connection details, the data is stored in .env
load_dotenv()
DB_NAME = os.getenv("DB_NAME") 
DB_USER = os.getenv("DB_USER") 
DB_PASSWORD = os.getenv("DB_PASSWORD") 
DB_HOST = os.getenv("DB_HOST") 
DB_PORT = os.getenv("DB_PORT")

#function to store data in database
def store_emotion(emotion, date, time, confidence = None):
    insert_query = """
    INSERT INTO emotions (emotion, date, time, confidence)
    VALUES (%s, %s, %s, %s)
    """

    conn = None
    cursor = None

    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cursor = conn.cursor()
        cursor.execute(insert_query, (emotion, date, time, confidence))
        conn.commit()
        print(f"Data stored successfully: {emotion} at {date + time}")
    except psycopg2.Error as e:
        print(f"Database error: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


In [ ]:
# webcam script

# Emotion classes (in the same order as during training)
classes = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

# Load the pre-trained Haar Cascade Classifier for face detection
# full credit to opencv for this model that is used to recognise faces from the images
# https://github.com/opencv/opencv/tree/master
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start webcam
cap = cv2.VideoCapture(0)  # 0 for default camera; use 1 if an external webcam is used

if not cap.isOpened():
    raise IOError("Cannot open webcam")

# font for text display
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_thickness = 2
rectangle_bgr = (255, 255, 255)  

# Time tracking for delay
last_processed_time = time.time()  # Initialize to the current time
process_interval = 5  # Interval in seconds

while True:
    # Read frame from webcam
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Get the current time
    current_time = time.time()

    if current_time - last_processed_time >= process_interval:
        # Update the last processed time
        last_processed_time = current_time
    
        # Convert frame to grayscale for Haar Cascade
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
        # process the detected faces
        for (x, y, w, h) in faces:
            # Extract the face ROI (Region of Interest)
            face_roi = frame[y:y + h, x:x + w]
            
            # Preprocess the face image for the model
            face_resized = cv2.resize(face_roi, (224, 224))  # Resize to model's input size
            face_normalized = face_resized.astype('float32') / 255.0  # Normalize pixel values to [0, 1] since this is how the model that recognises emotion was trained
            face_input = np.expand_dims(face_normalized, axis=0)  # Add batch dimension
    
            # Predict the emotion
            emotion_probabilities = model.predict(face_input)
            emotion_index = np.argmax(emotion_probabilities)  # Get the index of the highest probability
            predicted_emotion = classes[emotion_index]  # Map the index to the corresponding label
            confidence = float(emotion_probabilities[0][emotion_index])
    
            # Save the detected emotion to the database
            now = datetime.now()
            store_emotion(
                emotion=predicted_emotion,
                date=now.strftime("%Y-%m-%d"),
                time=now.strftime("%H:%M:%S"),
                confidence=confidence
            )
    
            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue rectangle
            
            # Prepare text background
            (text_width, text_height), baseline = cv2.getTextSize(predicted_emotion, font, font_scale, font_thickness)
            text_x = x
            text_y = y - 10  # Position text above the face rectangle
            box_coords = ((text_x, text_y - text_height - 10), (text_x + text_width + 10, text_y + baseline - 10))
            cv2.rectangle(frame, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
    
            # Put the predicted emotion text on the frame
            cv2.putText(frame, predicted_emotion, (text_x, text_y), font, font_scale, (0, 0, 0), font_thickness)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close display windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Data stored successfully: surprise at 2024-12-1316:20:42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Data stored successfully: neutral at 2024-12-1316:20:46
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Data stored successfully: neutral at 2024-12-1316:20:51
